In [1]:
# First lets improve libraries that we are going to be used in this lab session
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from collections import Counter
import pickle as pkl
import random
import pdb
random.seed(134)
import pandas as pd
PAD_IDX = 0
UNK_IDX = 1
BATCH_SIZE = 32

In [2]:
train=pd.read_csv('hw2_data/snli_train.tsv',delimiter='\t',encoding='utf-8')
val=pd.read_csv('hw2_data/snli_val.tsv',delimiter='\t',encoding='utf-8')

In [3]:
train_targets=list(train['label'])
val_targets=list(val['label'])

In [4]:
val_targets= [0 if x=='entailment' else x for x in val_targets]
train_targets= [0 if x=='entailment' else x for x in train_targets]
val_targets= [1 if x=='neutral' else x for x in val_targets]
train_targets= [1 if x=='neutral' else x for x in train_targets]
val_targets= [2 if x=='contradiction' else x for x in val_targets]
train_targets= [2 if x=='contradiction' else x for x in train_targets]

In [5]:
train_sen1= list(train['sentence1'])
train_sen2= list(train['sentence2'])
val_sen1= list(val['sentence1'])
val_sen2= list(val['sentence2'])

In [6]:
a=[]
b=[]
for i in range(0,100000):
    tmp1=''
    tmp2=''
    for j in train_sen1[i].split(' '):
        tmp1 =tmp1 + ' ' + j
    for j in train_sen2[i].split(' '):
        tmp2=tmp2 + ' ' + j
    a.append(tmp1[1:])
    b.append(tmp2[1:])

In [7]:
train_sen1= a
train_sen2= b

In [8]:
a=[]
b=[]
for i in range(0,1000):
    tmp1=''
    tmp2=''
    for j in val_sen1[i].split(' '):
        tmp1 =tmp1 + ' ' + j
    for j in val_sen2[i].split(' '):
        tmp2=tmp2 + ' ' + j
    a.append(tmp1[1:])
    b.append(tmp2[1:])

In [9]:
val_sen1= a
val_sen2= b

In [10]:
# Let's write the tokenization function 
# set a hyperparameter - vocab size of dataset
#Takes most frequent 10000 words from training set and makes it a vocabulary
#Other words are put as unknown token.
#One for unknown and one for padding and 9998 for most frequent words
#Spacy will be used for tokenisation
import spacy
import string

# Load English tokenizer, tagger, parser, NER and word vectors
tokenizer = spacy.load('en_core_web_sm')

def tokenize(sent):
  tokens = tokenizer(sent)
  return [token.text for token in tokens]



In [11]:
def tokenize_dataset(dataset):
    token_dataset = []
    # we are keeping track of all tokens in dataset 
    # in order to create vocabulary later

    for sample in dataset:
        tokens = tokenize(sample)
        
        token_dataset.append(tokens)
        
    return token_dataset

In [12]:
tt_sen1=tokenize_dataset(train_sen1)
tt_sen2=tokenize_dataset(train_sen2)
tv_sen1=tokenize_dataset(val_sen1)
tv_sen2=tokenize_dataset(val_sen2)

In [13]:
import pickle as pkl
pkl.dump(tt_sen1, open("tt_sen1.p", "wb"))
pkl.dump(tt_sen2, open("tt_sen2.p", "wb"))
pkl.dump(tv_sen1, open("tv_sen1.p", "wb"))
pkl.dump(tv_sen2, open("tv_sen2.p", "wb"))
pkl.dump(train_targets, open("train_targets.p", "wb"))
pkl.dump(val_targets, open("val_targets.p", "wb"))

In [2]:
import pickle as pkl
tt_sen1 = pkl.load(open("tt_sen1.p", "rb"))
tt_sen2 = pkl.load(open("tt_sen2.p", "rb"))
tv_sen1 = pkl.load(open("tv_sen1.p", "rb"))
tv_sen2 = pkl.load(open("tv_sen2.p", "rb"))
train_targets = pkl.load(open("train_targets.p", "rb"))
val_targets = pkl.load(open("val_targets.p", "rb"))

In [3]:
words_to_load = 50000

import numpy as np

with open('wiki-news-300d-1M.vec') as f:
    loaded_embeddings_ft = np.zeros((words_to_load, 300))
    words_ft = {}
    idx2words_ft = {}
    ordered_words_ft = []
    for i, line in enumerate(f):
        if i >= words_to_load: 
            break
        s = line.split()
        loaded_embeddings_ft[i, :] = np.asarray(s[1:])
        words_ft[s[0]] = i
        idx2words_ft[i] = s[0]
        ordered_words_ft.append(s[0])

In [4]:
#id2token
x=idx2words_ft
#token2idx
y=words_ft

In [5]:
idx2words_ft={}
idx2words_ft[0]='<pad>'
idx2words_ft[1]='<unk>'
for i in range(0,50000):
    idx2words_ft[i+2]=x[i]

In [6]:
w=list(words_ft.keys())
words_ft={}
words_ft['<pad>']=0
words_ft['<unk>']=1
for i in range(0,50000):
    words_ft[w[i]]=i+2

In [7]:
z=loaded_embeddings_ft

In [8]:
loaded_embeddings_ft=np.zeros(300)

In [9]:
loaded_embeddings_ft=np.vstack([loaded_embeddings_ft,np.ones(300)])

In [10]:
loaded_embeddings_ft=np.vstack([loaded_embeddings_ft,z])

In [11]:
token2id=words_ft

In [12]:
id2token=list(idx2words_ft.values())

In [13]:
# convert token to id in the dataset
def token2index_dataset(tokens_data):  #REplaces each token with respective index
    indices_data = []
    for tokens in tokens_data:
        index_list = [token2id[token] if token in token2id else UNK_IDX for token in tokens]
        indices_data.append(index_list)
    return indices_data

train_data_indices1 = token2index_dataset(tt_sen1)
train_data_indices2 = token2index_dataset(tt_sen2)
val_data_indices1 = token2index_dataset(tv_sen1)
val_data_indices2 = token2index_dataset(tv_sen2)

# double checking
print ("Train dataset size is {}".format(len(train_data_indices1)))
print ("Train dataset size is {}".format(len(train_data_indices2)))
print ("Val dataset size is {}".format(len(val_data_indices1)))
print ("Val dataset size is {}".format(len(val_data_indices2)))

Train dataset size is 100000
Train dataset size is 100000
Val dataset size is 1000
Val dataset size is 1000


In [14]:
mm=0
for i in range(0,100000):
    r=max(len(train_data_indices1[i]),len(train_data_indices2[i]))
    if mm<r:
        mm=r

In [15]:
mm

82

In [16]:
import numpy as np
import torch
from torch.utils.data import Dataset

class NewsGroupDataset(Dataset):
    """
    Class that represents a train/validation/test dataset that's readable for PyTorch
    Note that this class inherits torch.utils.data.Dataset
    """    
    def __init__(self, data_list1,data_list2, target_list):
        """
        @param data_list: list of newsgroup tokens 
        @param target_list: list of newsgroup targets 

        """
        self.data_list1 = data_list1
        self.data_list2 = data_list2
        self.target_list = target_list
        assert (len(self.data_list1) == len(self.target_list))
        assert (len(self.data_list2) == len(self.target_list))
        assert (len(self.data_list1) == len(self.data_list2))
    def __len__(self):
        assert (len(self.data_list1) == len(self.data_list2))
        return len(self.data_list1)
        
    def __getitem__(self, key):
        """
        Triggered when you call dataset[i]
        """
        token_idx1 = self.data_list1[key][:MAX_SENTENCE_LENGTH]       
        token_idx2 = self.data_list2[key][:MAX_SENTENCE_LENGTH]
        label = self.target_list[key]
        return [token_idx1, len(token_idx1),token_idx2, len(token_idx2), label]

In [17]:
MAX_SENTENCE_LENGTH = mm
#MAX_SENTENCE_LENGTH is a hyperparameter
#We implement dataset first before data loader. It takes 2 things as input.
#Datatlist (dataset converted to indices of tokens)
#Targetlist ( number between 1-20 that represents the target of document)
#We need to implement len and getitem

import numpy as np
import torch
from torch.utils.data import Dataset

class NewsGroupDataset(Dataset):
    """
    Class that represents a train/validation/test dataset that's readable for PyTorch
    Note that this class inherits torch.utils.data.Dataset
    """    
    def __init__(self, data_list1,data_list2, target_list):
        """
        @param data_list: list of newsgroup tokens 
        @param target_list: list of newsgroup targets 

        """
        self.data_list1 = data_list1
        self.data_list2 = data_list2
        self.target_list = target_list
        assert (len(self.data_list1) == len(self.target_list))
        assert (len(self.data_list2) == len(self.target_list))
        assert (len(self.data_list1) == len(self.data_list2))
    def __len__(self):
        assert (len(self.data_list1) == len(self.data_list2))
        return len(self.data_list1)
        
    def __getitem__(self, key):
        """
        Triggered when you call dataset[i]
        """
        token_idx1 = self.data_list1[key][:MAX_SENTENCE_LENGTH]       
        token_idx2 = self.data_list2[key][:MAX_SENTENCE_LENGTH]
        label = self.target_list[key]
        return [token_idx1, len(token_idx1),token_idx2, len(token_idx2), label]
#Collate function adds padding symbols to data in case its smaller than
# the max sentence length
def newsgroup_collate_func(batch):
    """
    Customized function for DataLoader that dynamically pads the batch so that all 
    data have the same length
    """
    data_list1 = []
    length_list1 = []
    data_list2 = []
    length_list2 = []
    label_list = []

    #print("collate batch: ", batch[0][0])
    #batch[0][0] = batch[0][0][:MAX_SENTENCE_LENGTH]
    for datum in batch:
        label_list.append(datum[4])
        length_list1.append(datum[1])
        length_list2.append(datum[3])
    # padding
    for datum in batch:
        padded_vec1 = np.pad(np.array(datum[0]), 
                                pad_width=((0,MAX_SENTENCE_LENGTH-datum[1])), 
                                mode="constant", constant_values=0)
        data_list1.append(padded_vec1)
        padded_vec2 = np.pad(np.array(datum[2]), 
                                pad_width=((0,MAX_SENTENCE_LENGTH-datum[3])), 
                                mode="constant", constant_values=0)
        data_list2.append(padded_vec2)
        
#     ind_dec_order1 = np.argsort(length_list1)[::-1]
#     ind_dec_order2 = np.argsort(length_list2)[::-1]
    
#     data_list1 = np.array(data_list1)[ind_dec_order1]
#     data_list2 = np.array(data_list2)[ind_dec_order2]
    
#     length_list1 = np.array(length_list1)[ind_dec_order1]
#     length_list2 = np.array(length_list2)[ind_dec_order2]
    
#     label_list = np.array(label_list)[ind_dec_order]    
        
    return [torch.from_numpy(np.array(data_list1)), torch.from_numpy(np.array(np.int32(length_list1))) ,torch.from_numpy(np.array(data_list2)), torch.from_numpy(np.array(np.int32(length_list2))), torch.from_numpy(np.array(np.int32(label_list))).long()]

# create pytorch dataloader
#train_loader = NewsGroupDataset(train_data_indices, train_targets)
#val_loader = NewsGroupDataset(val_data_indices, val_targets)
#test_loader = NewsGroupDataset(test_data_indices, test_targets)
#train_dataset is a hyperparameter and also batchsize
#train and validation also has shuffling here
BATCH_SIZE = 32
train_dataset = NewsGroupDataset(train_data_indices1, train_data_indices2, train_targets)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=BATCH_SIZE,
                                           collate_fn=newsgroup_collate_func,
                                           shuffle=True)


val_dataset = NewsGroupDataset(val_data_indices1, val_data_indices2, val_targets)
val_loader = torch.utils.data.DataLoader(dataset=val_dataset, 
                                           batch_size=BATCH_SIZE,
                                           collate_fn=newsgroup_collate_func,
                                           shuffle=True)


#for i, (data, lengths, labels) in enumerate(train_loader):
#    print (data)
#    print (labels)
#    break

### Now lets implement basic Convolutional Neural Net model for text


In [18]:
weight = torch.FloatTensor(loaded_embeddings_ft)

In [19]:
class CNN(nn.Module):
    def __init__(self, emb_size, hidden_size, num_layers, num_classes, vocab_size):

        super(CNN, self).__init__()

        self.num_layers, self.hidden_size = num_layers, hidden_size
        self.embedding = nn.Embedding.from_pretrained(weight,freeze=True)
    
        self.conv1 = nn.Conv1d(emb_size, hidden_size, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(hidden_size, hidden_size, kernel_size=3, padding=1)

        self.linear = nn.Linear(2*hidden_size, num_classes)

    def forward(self, x1, lengths1,x2, lengths2):
        batch_size1, seq_len1 = x1.size()
        batch_size2, seq_len2 = x2.size()
        
        embed1 = self.embedding(x1)
        embed2 = self.embedding(x2)        
        #First sentence
        hidden1 = self.conv1(embed1.transpose(1,2)).transpose(1,2)
        hidden1 = F.relu(hidden1.contiguous().view(-1, hidden1.size(-1))).view(batch_size1, seq_len1, hidden1.size(-1))

        hidden1 = self.conv2(hidden1.transpose(1,2)).transpose(1,2)
        hidden1 = F.relu(hidden1.contiguous().view(-1, hidden1.size(-1))).view(batch_size1, seq_len1, hidden1.size(-1))
        hidden1 = torch.sum(hidden1, dim=1)
       
        
        #second sentence
        hidden2 = self.conv1(embed2.transpose(1,2)).transpose(1,2)
        hidden2 = F.relu(hidden2.contiguous().view(-1, hidden2.size(-1))).view(batch_size2, seq_len2, hidden2.size(-1))

        hidden2 = self.conv2(hidden2.transpose(1,2)).transpose(1,2)
        hidden2 = F.relu(hidden2.contiguous().view(-1, hidden2.size(-1))).view(batch_size2, seq_len2, hidden2.size(-1))
        hidden2 = torch.sum(hidden2, dim=1)
        
        hiddenfinal = torch.cat((hidden1, hidden2), 1)
        logits = self.linear(hiddenfinal)
        return logits

In [20]:
llist=[]
pl=[]
pa=[]
epl=[]
epa=[]
def test_model(loader, model):
    """
    Help function that tests the model's performance on a dataset
    @param: loader - data loader for the dataset to test against
    """
    correct = 0
    total = 0
    model.eval()
    for data1, lengths1,data2, lengths2, labels in loader:
        data_batch1, lengths_batch1,data_batch2, lengths_batch2, label_batch = data1, lengths1,data2, lengths2, labels
        outputs = F.softmax(model(data_batch1, lengths_batch1,data_batch2, lengths_batch2), dim=1)
        predicted = outputs.max(1, keepdim=True)[1]

        total += labels.size(0)
        correct += predicted.eq(labels.view_as(predicted)).sum().item()
        llist.append([data1,data2,labels,predicted])
    return (100 * correct / total)


model = CNN(emb_size=300, hidden_size=600, num_layers=2, num_classes=3, vocab_size=50002)

learning_rate = 3e-4
num_epochs = 10 # number epoch to train

# Criterion and Optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
total_step = len(train_loader)

for epoch in range(num_epochs):
    l=0
    a=0
    c=0
    for i, (data1, lengths1, data2, lengths2, labels) in enumerate(train_loader):
        model.train()
        optimizer.zero_grad()
        # Forward pass
        outputs = model(data1, lengths1,data2, lengths2)
        loss = criterion(outputs, labels)

        # Backward and optimize
        loss.backward()
        optimizer.step()
        # validate every 100 iterations
        if i > 0 and i % 100 == 0:
            # validate
            val_acc = test_model(val_loader, model)
            c=c+1
            l=l+loss.item()
            a=a+val_acc
            print('Epoch: [{}/{}], Step: [{}/{}], Validation Acc: {}'.format(
                       epoch+1, num_epochs, i+1, len(train_loader), val_acc))
            pa.append(val_acc)
            pl.append(loss.item())
    epl.append(l/c)
    epa.append(a/c)

Epoch: [1/10], Step: [101/3125], Validation Acc: 46.7
Epoch: [1/10], Step: [201/3125], Validation Acc: 53.9
Epoch: [1/10], Step: [301/3125], Validation Acc: 55.9
Epoch: [1/10], Step: [401/3125], Validation Acc: 59.2
Epoch: [1/10], Step: [501/3125], Validation Acc: 58.8
Epoch: [1/10], Step: [601/3125], Validation Acc: 57.1
Epoch: [1/10], Step: [701/3125], Validation Acc: 57.9
Epoch: [1/10], Step: [801/3125], Validation Acc: 60.9
Epoch: [1/10], Step: [901/3125], Validation Acc: 59.9
Epoch: [1/10], Step: [1001/3125], Validation Acc: 59.2
Epoch: [1/10], Step: [1101/3125], Validation Acc: 60.2
Epoch: [1/10], Step: [1201/3125], Validation Acc: 59.8
Epoch: [1/10], Step: [1301/3125], Validation Acc: 58.9
Epoch: [1/10], Step: [1401/3125], Validation Acc: 60.1
Epoch: [1/10], Step: [1501/3125], Validation Acc: 61.2
Epoch: [1/10], Step: [1601/3125], Validation Acc: 62.3
Epoch: [1/10], Step: [1701/3125], Validation Acc: 62.9
Epoch: [1/10], Step: [1801/3125], Validation Acc: 62.1
Epoch: [1/10], Step

Epoch: [5/10], Step: [2701/3125], Validation Acc: 64.8
Epoch: [5/10], Step: [2801/3125], Validation Acc: 63.4
Epoch: [5/10], Step: [2901/3125], Validation Acc: 63.4
Epoch: [5/10], Step: [3001/3125], Validation Acc: 63.8
Epoch: [5/10], Step: [3101/3125], Validation Acc: 63.3
Epoch: [6/10], Step: [101/3125], Validation Acc: 64.1
Epoch: [6/10], Step: [201/3125], Validation Acc: 64.9
Epoch: [6/10], Step: [301/3125], Validation Acc: 63.8
Epoch: [6/10], Step: [401/3125], Validation Acc: 61.6
Epoch: [6/10], Step: [501/3125], Validation Acc: 62.0
Epoch: [6/10], Step: [601/3125], Validation Acc: 63.2
Epoch: [6/10], Step: [701/3125], Validation Acc: 62.9
Epoch: [6/10], Step: [801/3125], Validation Acc: 62.8
Epoch: [6/10], Step: [901/3125], Validation Acc: 64.7
Epoch: [6/10], Step: [1001/3125], Validation Acc: 64.7
Epoch: [6/10], Step: [1101/3125], Validation Acc: 61.7
Epoch: [6/10], Step: [1201/3125], Validation Acc: 62.4
Epoch: [6/10], Step: [1301/3125], Validation Acc: 63.8
Epoch: [6/10], Step

Epoch: [10/10], Step: [2201/3125], Validation Acc: 63.4
Epoch: [10/10], Step: [2301/3125], Validation Acc: 62.0
Epoch: [10/10], Step: [2401/3125], Validation Acc: 62.8
Epoch: [10/10], Step: [2501/3125], Validation Acc: 62.7
Epoch: [10/10], Step: [2601/3125], Validation Acc: 62.4
Epoch: [10/10], Step: [2701/3125], Validation Acc: 64.1
Epoch: [10/10], Step: [2801/3125], Validation Acc: 63.8
Epoch: [10/10], Step: [2901/3125], Validation Acc: 64.0
Epoch: [10/10], Step: [3001/3125], Validation Acc: 64.0
Epoch: [10/10], Step: [3101/3125], Validation Acc: 63.5


In [21]:
epa

[60.15806451612904,
 63.62258064516128,
 64.36451612903225,
 64.33225806451614,
 64.29354838709678,
 63.490322580645156,
 63.27096774193548,
 63.00645161290323,
 62.54193548387097,
 62.73870967741936]

In [22]:
epl

[0.8360626601403759,
 0.7915611267089844,
 0.7235932157885644,
 0.6791041022346865,
 0.5987347585539664,
 0.5031779129658976,
 0.4353878550952481,
 0.3277960462916282,
 0.32129123662748643,
 0.24950919732932123]

In [23]:
max(epa)

64.36451612903225